In [1]:
import json
import pandas as pd

# data clean
import re # punctuation,data clean

# tokenization and feature extraction
# install expacted word bag for jieba
import jieba 
jieba.load_userdict('jieba_expanded_dict.txt')
import jieba.analyse # TF-IDF 
import numpy as np # TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF

# models
from sklearn.ensemble import RandomForestClassifier # RF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC # SVM
from sklearn.neural_network import MLPClassifier # MLP
from sklearn.model_selection import RandomizedSearchCV # MLP

# analyse
import seaborn as sns # density curve 
import matplotlib.pyplot as plt # pie chart

from sklearn.model_selection import cross_validate

C:\Users\lenovo\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lenovo\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lenovo\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.446 seconds.
Prefix dict has been built successfully.
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
balanced_df_all_cleaned_tokenization = pd.read_csv('balanced_df_all_cleaned_tokenization.csv')

In [3]:
balanced_df_all_cleaned_tokenization2 = balanced_df_all_cleaned_tokenization.copy()

In [4]:
balanced_df_all_cleaned_tokenization2

,ID,label,tweet_content
0,25640,0,害真 挺 吃 颜想 岁 足 表达 减肥 决心 姐妹 谈恋爱 感觉 全世界 闺蜜 介绍 成功 ...
1,28303,0,拥抱 世界 暖心 动作 生活 中 难免 挫折 坎坷 安 慌乱 许 拥抱 充满 量 世界 抗衡...
2,10546,0,雨天 穿 滑倒 坐水里 亲测 艾特子 赟 家 意见 麻雀 妈妈 问 麻雀 扎 头发 麻雀 说...
3,26929,0,幸福 做 桌菜 然家 朋友 吃 完 没 帮忙 刷碗 老规矩 昨晚 想 做 早餐 奶包 发酵 ...
4,31080,0,日 清明 青草 疫情 中 牺牲 医护 员 公安干警 基层干部 线 工作 逝世 胞 沉痛 悼念...
...,...,...,...
20645,10321,1,意做件 错事 真 久久 原谅 近 发生 事 情绪 波动 房间 爆哭然 雨天 跑 出 外面 冻...
20646,10322,1,太 喜欢 张 教授 说话 特实 危言耸听 然 整体 医生 亲戚朋友 医生 医生 子女 微博 ...
20647,10323,1,毒药 断送 爱 听说 爱情 蜜糖 深陷 沉醉 岂知 份 爱情 美意 想 原理 结束 意想 称...
20648,10324,1,果 原谅 痛苦 值 降低 贼 悔 吃 麻辣 串 洗完 澡 新 换衣服 全味 难受 新闻 男 ...


In [5]:
# 取50行label为0，50行label为1
df_label_0 = balanced_df_all_cleaned_tokenization2[balanced_df_all_cleaned_tokenization2['label'] == 0].sample(n=50, random_state=42)
df_label_1 = balanced_df_all_cleaned_tokenization2[balanced_df_all_cleaned_tokenization2['label'] == 1].sample(n=50, random_state=42)

# 合并两个dataframe
balanced_df_all_cleaned_tokenization2 = pd.concat([df_label_0, df_label_1])

print(balanced_df_all_cleaned_tokenization2)

          ID  label                                      tweet_content
10148  31839      0  手中 行 原油 宝 失误 客户 买单 国油板 价想 生日 月 日 祝福 事 花开花落 难堪 ...
1692   22295      0  北极 绒纯 棉裤 条卷 评拍 送条 共 发条 肠道 提高 孩子 免疫力 卷 评晚 睡觉 抱 ...
2692   17168      0  早安 周季 松茸 价格低 家斤 囤 优惠 特价 家 囤 祝 生日快乐 喝点 水朵 姣 云南 ...
7703   21714      0  式样 友纸 召唤 训导 快 平安 京 富豪 灯烬 熄烛 阴阳 全新 阶式 神 浮世 青行 灯...
321    29150      0  做 学习型 妈妈 建设 学习 成长型 家庭 建立 家庭 精神 宝藏 免费 围观 回答 价值 ...
...      ...    ...                                                ...
13388   3064      1  十 症状 否患 抑郁症 情绪 抑郁 抑郁 情绪 泣 成声 抑郁症 症状 然 抑郁症 患 感觉...
17587   7263      1  确保 勇气 步原 轨迹 走次 想 杀 想 爸爸 难 告诉 熬 熬 熬 熬 睡觉 件 深刻 体...
13695   3371      1  红枣 熬汤 止 润肺 取 银耳 克 红枣 颗加 冰糖 煮 半时 红枣 煮蛋 补血 养颜 红枣...
13812   3488      1  悔 万分 难受 找 动摇 重蹈覆辙 想 原谅 周末 忙刻 闲 整场 活动 撑气 想 骂 洞察...
10636    312      1  班族 晚班 步行 回家 间歇 跳绳 分钟 想 减肥 女性 摇呼圈 分钟 晚 仰卧 坐 男性 ...

[100 rows x 3 columns]


In [6]:
# replace NaN values with an empty string
balanced_df_all_cleaned_tokenization2['tweet_content'] = balanced_df_all_cleaned_tokenization2['tweet_content'].fillna('')

In [7]:
# create a TfidfVectorizer object 将tweet_content列作为特征向量化
vectorizer = TfidfVectorizer()

In [8]:
# fit and transform the data
X = vectorizer.fit_transform(balanced_df_all_cleaned_tokenization2['tweet_content'])

In [9]:
# 将label列作为目标变量
y = balanced_df_all_cleaned_tokenization2['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## test e_estimatiors


In [10]:
# 创建一个随机森林分类器
rfc = RandomForestClassifier(random_state=66)

In [11]:
# 使用cross_validate进行交叉验证，并计算多个评估指标，此时不需要划分测试集和训练集
scoring = ['accuracy', 'precision', 'recall', 'f1']
cv_results = cross_validate(rfc, X, y, cv=10, scoring=scoring)

# 输出每次交叉验证的结果
print("Cross-validation results:")
for metric in scoring:
    print(f"{metric.capitalize()}: {cv_results[f'test_{metric}']}")

# 输出平均结果
print("Average Accuracy:", cv_results['test_accuracy'].mean())
print("Average Precision:", cv_results['test_precision'].mean())
print("Average Recall:", cv_results['test_recall'].mean())
print("Average F1 Score:", cv_results['test_f1'].mean())

Cross-validation results:
Accuracy: [0.7 1.  0.9 1.  0.8 1.  0.9 0.9 0.9 1. ]
Precision: [1.  1.  1.  1.  0.8 1.  1.  1.  1.  1. ]
Recall: [0.4 1.  0.8 1.  0.8 1.  0.8 0.8 0.8 1. ]
F1: [0.57142857 1.         0.88888889 1.         0.8        1.
 0.88888889 0.88888889 0.88888889 1.        ]
Average Accuracy: 0.9099999999999999
Average Precision: 0.9800000000000001
Average Recall: 0.8400000000000001
Average F1 Score: 0.8926984126984128


In [16]:
# 使用10折交叉验证来确定n_estimators的最佳值
scores = []
for n in range(1, 101):
    rf.set_params(n_estimators=n)
    score = cross_val_score(rf, X_train, y_train, cv=10).mean()
    scores.append(score)

In [17]:
# 找到具有最佳性能的模型
best_n = np.argmax(scores) + 1

In [18]:
# 使用测试集来评估具有最佳性能的模型的泛化能力
rf.set_params(n_estimators=best_n)
rf.fit(X_train, y_train)
test_score = rf.score(X_test, y_test)


In [19]:
# Print out the best n_estimators value
print("The best n_estimators value is:", best_n)

The best n_estimators value is: 13


In [20]:
test_score

0.8